## Script used to extract dataset information from ENCODE for ChIP, ATAC, and DNase-seq

#### General structure of ENCODE search filtering:
1. Search ENCODE for list of experiments meeting criteria
2. Extract experiment metadata and filter for presence of controls (ChIP only)
3. Iterate through files of experiment, filter, and categorize into paired-end or single-end sequenced
4. Write files

In [1]:
import requests
import json
from time import sleep

headers = {'accept': 'application/json'}

In [2]:
# Simple filtering function that works by concatenating filter terms and values to ENCODE's URL
# Can pass a dictionary, or works with kwargs - added support for kwargs for some filters with periods in their names

def apply_filters_to_encode_search_url(encode_url, **kwargs):
    for key, value in kwargs.items():
        if key == 'biosample':
            key = 'biosample_ontology.term_name'
        if key == 'audit':
            key = 'audit.WARNING.category'
        if key == 'target':
            key = 'target.label'
        if key == 'target_class':
            key = 'target.investigated_as'
        if value.startswith('!'):
            encode_url = encode_url+'&{}!={}'.format(key, value[1:])
        else:
            encode_url = encode_url+'&{}={}'.format(key, value)
    return encode_url

### ChIP functions

In [3]:
# Function used for finding ENCODE experiments that have a control track and a target (used for ChIP)
# The experiment accession number, control accession number, target, and biosample info are extracted from search results
# All provided information on biosample is compressed into a continuous string without spaces

def make_experiment_accession_control_list(experiment_search_results_dict):
    experiment_accession_list = []
    for experiment_dict in experiment_search_results_dict:
        if experiment_dict['possible_controls']:
            attributes_dict = {}
            control_accession_path = experiment_dict['possible_controls'][0]
            control_accession = control_accession_path[1:-1].split('/')[1]
            target_name_path = experiment_dict['target']
            target_name = target_name_path[1:-1].split('/')[1]
            biosample_string = (experiment_dict['biosample_summary'])
            alnum_biosample_string = ''.join(x for x in biosample_string if x.isalnum()).replace('μ', 'mc').replace('β', 'beta').replace('α', 'alpha').replace('γ', 'gamma')
            attributes_dict['experiment_accession'] = experiment_dict['accession']
            attributes_dict['control_accession'] = control_accession
            attributes_dict['target'] = target_name
            attributes_dict['biosample'] = alnum_biosample_string
            experiment_accession_list.append([experiment_dict['accession'], attributes_dict])

    return experiment_accession_list

In [4]:
# Loops through list of samples in an experiment, and extracts bam file accession numbers
# Separates paired and single-end experiments and returns as two separate lists of bam file accession numbers
# Info on paired vs single end reads is in notes/qc/qc/ from the @graph dict from ENCODE's file search content; if this information was missing, the sample was disregarded
# Filters applied: status=released; genome=hg38; and must have metadata regarding single/paired-end sequencing
# pe_bam_list contains only the accession number of the bam; se_bam_list also includes the estimated fragment length neccessary for downstream processing

def extract_samples_from_experiment_dna_binding(experiment_accession_num):
    experiment_files_search_url = 'https://www.encodeproject.org/search/?type=File&dataset=/experiments/{}/&format=json&frame=object&limit=all'.format(experiment_accession_num)
    filtered_experiment_files_search_url = apply_filters_to_encode_search_url(experiment_files_search_url, status='released', assembly='GRCh38', output_type='alignments')
    response = requests.get(filtered_experiment_files_search_url, headers=headers)
    sleep(1)
    experiment_files_search_results = response.json()
    pe_bam_list = []
    se_bam_list = []
    for file_data_dict in experiment_files_search_results['@graph']:
        if ('file_type' and 'no_file_available' and 'notes') in file_data_dict:
            if (file_data_dict['file_type']=='bam') and (file_data_dict['no_file_available']==False):
                notes_dict = json.loads(file_data_dict['notes'])
                if 'qc' in notes_dict:
                    if ('qc' and 'xcor_qc') in notes_dict['qc']:
                        if 'properly_paired' in notes_dict['qc']['qc']:
                            if notes_dict['qc']['qc']['properly_paired'][0]>0:
                                pe_bam_list.append(file_data_dict['accession'])
                            else:
                                if 'estFragLen' in notes_dict['qc']['xcor_qc']:
                                    if int(notes_dict['qc']['xcor_qc']['estFragLen'])>0:
                                        se_bam_list.append([file_data_dict['accession'], notes_dict['qc']['xcor_qc']['estFragLen']])
    return pe_bam_list, se_bam_list

In [5]:
# Loops through list of experiments (generated from make_experiment_accession_control_list) and extracts bam files from individual samples
# Separates paired and single-end experiments using extract_samples_from_experiment_dna_binding function
# pe_bam_list_total contains only the accession number of the bam and its control; se_bam_list_total also includes the estimated fragment length for experiment and control bams

def generate_bam_file_lists_dna_binding(experiment_accession_list):
    pe_bam_list_total = []
    se_bam_list_total = []
    for experiment_accession_num, experiment_attributes_dict in experiment_accession_list:
        pe_bam_list, se_bam_list = extract_samples_from_experiment_dna_binding(experiment_accession_num)
        pe_bam_list_control, se_bam_list_control = extract_samples_from_experiment_dna_binding(experiment_attributes_dict['control_accession'])
        if pe_bam_list and pe_bam_list_control:
            for bam_accession in pe_bam_list:
                bam_attributes_dict = {}
                bam_attributes_dict['bam_accession'] = bam_accession
                bam_attributes_dict['bam_accession_control'] = pe_bam_list_control[0]
                pe_bam_list_total.append([bam_accession, bam_attributes_dict, experiment_attributes_dict])
        elif se_bam_list and se_bam_list_control:
            for bam_accession, frag_len in se_bam_list:
                bam_attributes_dict = {}
                bam_attributes_dict['bam_accession'] = bam_accession
                bam_attributes_dict['fragment_length'] = frag_len
                bam_attributes_dict['bam_accession_control'] = se_bam_list_control[0][0]
                bam_attributes_dict['fragment_length_control'] = se_bam_list_control[0][1]
                se_bam_list_total.append([bam_accession, bam_attributes_dict, experiment_attributes_dict])
    return pe_bam_list_total, se_bam_list_total

### Open chromatin functions

In [6]:
# Function used for finding ENCODE experiments (used for open chromatin)
# The experiment accession number and biosample info are extracted from search results
# All provided information on biosample is compressed into a continuous string without spaces

def make_experiment_accession_list(experiment_search_results_dict):
    experiment_accession_list = []
    for experiment_dict in experiment_search_results_dict:
        attributes_dict = {}
        biosample_string = (experiment_dict['biosample_summary'])
        alnum_biosample_string = ''.join(x for x in biosample_string if x.isalnum()).replace('μ', 'mc').replace('β', 'beta').replace('α', 'alpha').replace('γ', 'gamma')
        attributes_dict['experiment_accession'] = experiment_dict['accession']
        attributes_dict['biosample'] = alnum_biosample_string
        experiment_accession_list.append([experiment_dict['accession'], attributes_dict])
    return experiment_accession_list

In [7]:
# Loops through list of experiments (generated from make_experiment_accession_list function) and extracts bam files from individual samples
# Separates paired and single-end experiments and returns as two separate lists of bam file accession numbers
# Filters applied: status=released; genome=hg38; and must have metadata regarding single/paired-end sequencing

def extract_samples_from_experiments_generate_bam_file_lists_open_chrom(experiment_accession_list):
    bam_accession_qc_list = []
    for experiment_accession_num, experiment_attributes_dict in experiment_accession_list:
        experiment_files_search_url = 'https://www.encodeproject.org/search/?type=File&dataset=/experiments/{}/&format=json&frame=object&limit=all'.format(experiment_accession_num)
        filtered_experiment_files_search_url = apply_filters_to_encode_search_url(experiment_files_search_url, status='released', assembly='GRCh38', output_type='alignments')
        response = requests.get(filtered_experiment_files_search_url, headers=headers)
        sleep(1)
        experiment_files_search_results = response.json()
        for file_data_dict in experiment_files_search_results['@graph']:
            if ('file_type' and 'no_file_available' and 'quality_metrics') in file_data_dict:
                if (file_data_dict['file_type']=='bam') and (file_data_dict['no_file_available']==False):
                    bam_accession_qc_list.append([file_data_dict['accession'], file_data_dict['quality_metrics'], experiment_attributes_dict])
    pe_bam_list = []
    se_bam_list = []
    for bam_accession, qc_list, experiment_attributes_dict in bam_accession_qc_list:
        for qc_url in qc_list:
            if qc_url.startswith('/complexity-xcorr-quality-metrics/'):
                bam_qc_url = 'https://www.encodeproject.org{}?format=json'.format(qc_url)
                response = requests.get(bam_qc_url)
                sleep(1)
                bam_qc = response.json()
                if 'paired-end' in bam_qc:
                    if bam_qc['paired-end']==True:
                        pe_bam_list.append([bam_accession, experiment_attributes_dict])
                    elif bam_qc['paired-end']==False:
                        if 'read length' in bam_qc:
                            if int(bam_qc['read length'])>0:
                                se_bam_list.append([bam_accession, bam_qc['read length'], experiment_attributes_dict])
    return pe_bam_list, se_bam_list

### Main loop

In [8]:
# assay_slims is not important for filtering per se but it is important for logic later in the script; 

experiment_filters_dict = {
    'ATAC':{'assay_slims':'DNA+accessibility', 'assay_title':'ATAC-seq', 'assembly':'GRCh38', 'status':'released'},
    'DNase':{'assay_slims':'DNA+accessibility', 'assay_title':'DNase-seq', 'assembly':'GRCh38', 'status':'released'},
    'H3K27ac':{'assay_slims':'DNA+binding', 'assay_title':'Histone+ChIP-seq', 'target':'H3K27ac', 'assembly':'GRCh38', 'status':'released'},
    'H3K4me3':{'assay_slims':'DNA+binding','assay_title':'Histone+ChIP-seq', 'target':'H3K4me3', 'assembly':'GRCh38', 'status':'released'},
    'TF':{'assay_slims':'DNA+binding', 'assay_title':'TF+ChIP-seq', 'target_class':'transcription+factor', 'assembly':'GRCh38', 'status':'released'}
}

In [9]:
# it is assumed that assay_slims == DNA+binding needs a control and DNA+accessibility does not
# experiment_search_results is a dictionary of all experiments meeting filtering criteria; experiments may contain multiple samples

experiment_search_url = 'https://www.encodeproject.org/search/?type=Experiment&frame=object&limit=all&format=json'
for experiment_type in experiment_filters_dict:
    filtered_experiment_search_url = apply_filters_to_encode_search_url(experiment_search_url, **experiment_filters_dict[experiment_type])
    response = requests.get(filtered_experiment_search_url, headers=headers)
    sleep(1)
    experiment_search_results = response.json()
    
    if experiment_filters_dict[experiment_type]['assay_slims']=='DNA+binding':
        experiment_accession_list = make_experiment_accession_control_list(experiment_search_results['@graph'])
        pe_bam_list, se_bam_list = generate_bam_file_lists_dna_binding(experiment_accession_list)
        with open('{}_pairedend_encode_datasets.tsv'.format(experiment_type), 'w') as f:
            f.write('bam_accession\tbam_accession_control\ttarget\tbiosample\texperiment_accession\n')
            for sample_accession, sample_data_dict, experiment_data_dict in pe_bam_list:
                f.write('{}\t{}\t{}\t{}\t{}\n'.format(
                    sample_data_dict['bam_accession'], sample_data_dict['bam_accession_control'], 
                    experiment_data_dict['target'], experiment_data_dict['biosample'], experiment_data_dict['experiment_accession']))
        with open('{}_singleend_encode_datasets.tsv'.format(experiment_type), 'w') as f:
            f.write('bam_accession\tfragment_length\tbam_accession_control\tfragment_length_control\ttarget\tbiosample\texperiment_accession\n')
            for sample_accession, sample_data_dict, experiment_data_dict in se_bam_list:
                f.write('{}\t{}\t{}\t{}\t{}\t{}\t{}\n'.format(
                    sample_data_dict['bam_accession'], sample_data_dict['fragment_length'], 
                    sample_data_dict['bam_accession_control'], sample_data_dict['fragment_length_control'], 
                    experiment_data_dict['target'], experiment_data_dict['biosample'], experiment_data_dict['experiment_accession']))            
    else:
        experiment_accession_list = make_experiment_accession_list(experiment_search_results['@graph'])
        pe_bam_list, se_bam_list = extract_samples_from_experiments_generate_bam_file_lists_open_chrom(experiment_accession_list)
        with open('{}_pairedend_encode_datasets.tsv'.format(experiment_type), 'w') as f:
            f.write('bam_accession\tbiosample\texperiment_accession\n')
            for sample_accession, experiment_data_dict in pe_bam_list:
                f.write('{}\t{}\t{}\n'.format(
                    sample_accession, experiment_data_dict['biosample'], experiment_data_dict['experiment_accession']))
        with open('{}_singleend_encode_datasets.tsv'.format(experiment_type), 'w') as f:
            f.write('bam_accession\tfragment_length\tbiosample\texperiment_accession\n')
            for sample_accession, sample_fragment_length, experiment_data_dict in se_bam_list:
                f.write('{}\t{}\t{}\t{}\n'.format(
                    sample_accession, sample_fragment_length,
                    experiment_data_dict['biosample'], experiment_data_dict['experiment_accession']))            